In [2]:
!pip -qqq install yt-dlp

In [3]:
from yt_dlp import YoutubeDL
import os
import re

In [ ]:
#!pip install moviepy
#!pip install pytube

In [8]:
from pytube import YouTube
from moviepy import VideoFileClip

In [9]:
def download_youtube_video(url, download_path='downloads_vdo/'):
    # Create directory if it doesn't exist
    if not os.path.exists(download_path):
        os.makedirs(download_path)

    # Download the YouTube video
    yt = YouTube(url)
    video = yt.streams.filter(only_video=False, file_extension='mp4').first()
    video_path = video.download(output_path=download_path)

    print(f"Video downloaded: {video_path}")
    return video_path

In [10]:
video_urls = ["https://www.youtube.com/watch?v=qP1JKWBBy80",
                "https://www.youtube.com/watch?v=_p58cZIHDG4"]

In [ ]:
#for video in video_urls:
    #download_youtube_video(video)

In [ ]:
video_path = "/home/pms/llm_project/mm_rag_esg_financial_project/downloads_vdo/Invest_with_Conscience_ ESG_Investing.mp4"
audio_path = os.path.splitext(video_path)[0] + '.mp3'

video_clip = VideoFileClip(video_path)
video_clip.audio.write_audiofile(audio_path,codec='mp3')
video_clip.close()


In [ ]:
# class YouTubeAudioDownloader:
#     def __init__(self, output_folder):
#         self.output_folder = os.path.abspath(output_folder)
#         self.audio_files_dict = {}

#     def get_safe_filename(self, filename):
#         safe_filename = re.sub(r'[^\w\-.]', '_', filename)
#         safe_filename = re.sub(r'_+', '_', safe_filename)
#         safe_filename = safe_filename[:50].strip('_')
#         return safe_filename

#     def download_audio(self, video_url):
#         try:
#             ydl_opts = {
#                 'format': 'bestaudio/best',
#                 'postprocessors': [{
#                     'key': 'FFmpegExtractAudio',
#                     'preferredcodec': 'mp3',
#                     'preferredquality': '192',
#                 }],
#                 'outtmpl': os.path.join(self.output_folder, '%(title)s.%(ext)s'),
#             }

#             with YoutubeDL(ydl_opts) as ydl:
#                 info = ydl.extract_info(video_url, download=True)
#                 filename = ydl.prepare_filename(info)
#                 base, ext = os.path.splitext(filename)
#                 new_file = base + '.mp3'

#             print(f"Audio file downloaded: {new_file}")
#             self.audio_files_dict[video_url] = new_file
#             return new_file
#         except Exception as e:
#             print(f"Error downloading audio from {video_url}: {str(e)}")
#             return None

#     def download_multiple_audios(self, video_urls):
#         for url in video_urls:
#             print(f"Processing video: {url}")
#             audio_file = self.download_audio(url)
#             if audio_file is None:
#                 print(f"Failed to download audio from video: {url}")
#         return self.audio_files_dict

In [ ]:
# downloader = YouTubeAudioDownloader(output_folder="./data")
# video_urls = ["https://www.youtube.com/watch?v=qP1JKWBBy80",
#                 "https://www.youtube.com/watch?v=_p58cZIHDG4"]

# # Download audios from multiple videos
# audio_files = downloader.download_multiple_audios(video_urls)

# print("Downloaded audio files:")
# for audio_file in audio_files:
#     print(audio_file)